# 🧮 Mini Calculator Project

**Course**: Python Final Project  
**Group**: Group 3  
**Team Members**: 
- Reihaneh Niknahad
- Thi Ngoc Hanh Nguyen

## Project Description
Our project is a **Mini Calculator with a simple UI built using Gradio**.  
It supports basic arithmetic operations (`+`, `-`, `*`, `/`) and also advanced ones (`x²`, `√x`).  
The program is divided into multiple files for better structure and reusability:
- **simple_calculator.py** – first console-based version
- **math_core.py** – core calculation functions
- **calculator.py** – UI with Gradio

## Peer Evaluation (for teacher & teammates)

- Comments from team members will be added here.  

In [1]:
"""
====================================================
 Mini Calculator Project - math_core.py
----------------------------------------------------
 Purpose: 
   - Contains all core math functions (add, subtract, multiply, divide, sqrt, power).
   - Designed to be reusable for different UIs.
==================================
"""
import math

def calculate(a, b, op):
    if op == '+': 
        return a + b
    if op == '-': 
        return a - b
    if op == '*': 
        return a * b
    if op == '/':
        if b == 0:
            raise ZeroDivisionError("Cannot divide by 0")
        return a / b
    if op == '^2':
        return a ** 2
    if op == 'sqrt':
        if a < 0:
            raise ValueError("Cannot take sqrt of negative number")
        return math.sqrt(a)
    raise ValueError("Does not support this operator")

In [2]:
"""
====================================================
 Mini Calculator Project - calculator.py
----------------------------------------------------
 Purpose:
   - Provides the User Interface using Gradio.
   - Connects UI buttons to functions in math_core.py.
====================================================
"""

import gradio as gr
from math_core import calculate  # core calculation logic

# Map pretty labels -> internal tokens
PRETTY_TO_TOKEN = {
    "√𝑥": "√",
    "𝑥²": "^2",
}

# -------------------------------
# 1. Validate input
# -------------------------------
def validate_input(state, btn):
    """Check invalid input cases before processing"""
    if state.endswith("√") and btn == "-":
        return "Err: Cannot take sqrt of negative number"
    if state == "" and btn == "-":
        return "Err: Cannot start with minus"
    if "√-" in state + btn or "√(-" in state + btn:
        return "Err: Cannot take sqrt of negative number"
    return None


# -------------------------------
# 2. Do the calculation
# -------------------------------
def do_calculate(state):
    """Extract operator, operands and compute result"""
    try:
        op = None
        for o in ["+", "-", "*", "/", "^2", "√"]:
            if o in state:
                op = o
                break

        if op is None:
            return "Err: Invalid expression"

        if op == "√":
            a = float(state.replace("√", ""))
            if a < 0:
                return "Err: Negative number under sqrt"
            return str(calculate(a, 0, "sqrt"))

        elif op == "^2":
            a = float(state.replace("^2", ""))
            return str(calculate(a, 0, "^2"))

        else:  # binary ops
            parts = state.split(op)
            if len(parts) != 2:
                return "Err: Invalid expression"
            a = float(parts[0].strip())
            b = float(parts[1].strip())
            return str(calculate(a, b, op))

    except Exception as e:
        return f"Err: {e}"


# -------------------------------
# 3. Handle special buttons
# -------------------------------
def handle_special_buttons(btn, state):
    """Handle C, Exit, ="""
    if btn == "C":
        return "", ""
    elif btn == "Exit":
        return state, "The application was stopped", True
    elif btn == "=":
        result = do_calculate(state)
        return state, result, False
    return None


# -------------------------------
# 4. Main click handler
# -------------------------------
def on_click(btn, state, stopped):
    # Map pretty labels to tokens
    btn = PRETTY_TO_TOKEN.get(btn, btn)

    if stopped:
        return state, gr.update(value="Stop the application", elem_classes="display-err"), True

    # Validate input first
    err = validate_input(state, btn)
    if err:
        return state, gr.update(value=err, elem_classes="display-err"), False

    # Handle special buttons
    special = handle_special_buttons(btn, state)
    if special:
        s, val, stop = special
        elem = "display-err" if "Err" in val or "stopped" in val else "display-num"
        return s, gr.update(value=val, elem_classes=elem), stop

    # Otherwise append new char
    new_state = state + btn
    return new_state, gr.update(value=new_state, elem_classes="display-num"), False


# -------------------------------
# 5. Build UI
# -------------------------------
def launch_ui():
    with gr.Blocks(css="""
        .btn { width:60px !important; height:60px !important; font-size:18px; flex:none !important; }
        .display-num textarea { width:400px !important; height:40px !important; font-size:20px; text-align:right; }
        .display-err textarea { width:400px !important; height:40px !important; font-size:18px; text-align:left; color:#ff5555; }
    """) as demo:
        gr.Markdown("### 🧮 Mini Calculator")

        state = gr.State("")
        stopped = gr.State(False)

        with gr.Row():
            with gr.Column(scale=0):  
                display = gr.Textbox(
                    label="Result", 
                    value="", 
                    interactive=False, 
                    lines=2,
                    max_lines=2,
                    elem_classes="display-num"
                )

        buttons = [
            ["7", "8", "9", "/"],
            ["4", "5", "6", "*"],
            ["1", "2", "3", "-"],
            ["0", ".", "=", "+"],
            ["√𝑥", "𝑥²", "C", "Exit"]
        ]

        for row in buttons:
            with gr.Row():
                for label in row:
                    btn = gr.Button(label, elem_classes="btn")
                    internal = PRETTY_TO_TOKEN.get(label, label)
                    btn.click(
                        on_click,
                        inputs=[gr.Textbox(value=internal, visible=False), state, stopped],
                        outputs=[state, display, stopped],
                        show_progress=False
                    )

    demo.launch()


if __name__ == "__main__":
    launch_ui()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\LG\anaconda3\envs\myenv\Lib\site-packages\gradio\queueing.py", line 745, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LG\anaconda3\envs\myenv\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LG\anaconda3\envs\myenv\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LG\anaconda3\envs\myenv\Lib\site-packages\gradio\blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\LG\anaconda3\envs\myenv\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_asyn